<a href="https://colab.research.google.com/github/AniudhNegi/Hate-Speech-Detection/blob/main/hate_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns

In [ ]:
data=pd.read_csv("/tweets_data.csv")

In [ ]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,cl,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
data.shape

(24783, 7)

In [ ]:
x=data['cl']
data.rename(columns={
    'tweet':'text',
    'cl':'category'},
    inplace=True)
data['category']=data['category'].map({0:'hate_speech',
                                       1:"offensive_language",
                                       2:"neither"})
data=pd.concat([data['text'],data['category'],x], axis=1)
data.rename(columns={'cl' : 'label'}, 
                    inplace=True)

data

,text,category,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,neither,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive_language,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive_language,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive_language,1
...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,offensive_language,1
24779,"you've gone and broke the wrong heart baby, an...",neither,2
24780,young buck wanna eat!!.. dat nigguh like I ain...,offensive_language,1
24781,youu got wild bitches tellin you lies,offensive_language,1


In [ ]:
data['category'].value_counts()

offensive_language    19190
neither                4163
hate_speech            1430
Name: category, dtype: int64

In [ ]:
X_train_, X_test, y_train_, y_test =train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.label.values,   
    
)


In [ ]:
X_train, X_val, y_train, y_val =train_test_split(
    data.loc[X_train_].index.values,
    data.loc[X_train_].label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.loc[X_train_].label.values,    
)


In [ ]:
data['data']=['not_set']*data.shape[0]
data.loc[X_train, 'data'] = 'train'
data.loc[X_test, 'data'] = 'test'
data.loc[X_val, 'data'] = 'val'
data.groupby(['category','label','data']).count()

text
category           label data        
hate_speech        0     test     143
                         train   1158
                         val      129
neither            2     test     416
                         train   3372
                         val      375
offensive_language 1     test    1920
                         train  15543
                         val     1727

In [ ]:
data_train=data.loc[data['data']=='train']
data_test=data.loc[data['data']=='test']
data_val=data.loc[data['data']=='val']

In [ ]:
data_tv= pd.concat([data_train,data_val],axis=0)

In [ ]:
data_tv[0:50:10]

,text,category,label,data
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language,1,train
13,""" bad bitches is the only thing that i like """,offensive_language,1,train
28,""" i need a trippy bitch who fuck on Hennessy """,offensive_language,1,train
39,""" like Snoop said in 94 we dont love these hoes """,offensive_language,1,train
53,""" young Pill Chamberlain these bitches love my...",offensive_language,1,train


In [ ]:
x1=data_tv.text.values
y1=data_tv.label.values

In [ ]:
max_f=20000
max_l=512

In [ ]:
x_token=text.Tokenizer(max_f)
x_token.fit_on_texts(list(x1))

In [ ]:
x_tokenized = x_token.texts_to_sequences(x1)
x_trainval=sequence.pad_sequences(x_tokenized,maxlen=max_l)

In [ ]:
x_test_token = x_token.texts_to_sequences(data_test.text.values)
x_t=sequence.pad_sequences(x_test_token,maxlen=max_l)

In [ ]:
embedding_dim =100
embeddings_index = dict()
f = open("/content/glove.6B.100d.txt", encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word]= coefs
f.close()
print(f'Found {len(embeddings_index)} word vectors')

Found 28113 word vectors


In [ ]:
emb_matrix = np.zeros((max_f,embedding_dim))

for word, index in x_token.word_index.items():
    if index>max_f-1:
        break
    else:
        emb_vector=embeddings_index.get(word)
        if emb_vector is not None:
            emb_matrix[index] = emb_vector

In [ ]:
y_trainval = tf.keras.utils.to_categorical(y1, num_classes=3)

In [ ]:
y_test = tf.keras.utils.to_categorical(data_test.label, num_classes=3)
y_trainval

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(max_f,embedding_dim,
                    embeddings_initializer=tf.keras.initializers.Constant(emb_matrix),
                    trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(64,2,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(64,2,padding='valid',activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(32,2,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(32,2,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(GlobalMaxPooling1D())

model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         2000000   
                                                                 
 dropout_4 (Dropout)         (None, None, 100)         0         
                                                                 
 conv1d_8 (Conv1D)           (None, None, 64)          12864     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, None, 64)          8256      
                                                                 
 max_pooling1d_9 (MaxPooling  (None, None, 64)         0         
 1D)                                                  

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
hist=model.fit(x_trainval,y_trainval,validation_split=0.2,epochs=7)

Epoch 1/7
558/558 [==============================] - 45s 79ms/step - loss: 0.6439 - accuracy: 0.7851 - val_loss: 0.5187 - val_accuracy: 0.8269
Epoch 2/7
558/558 [==============================] - 46s 82ms/step - loss: 0.5447 - accuracy: 0.8180 - val_loss: 0.4878 - val_accuracy: 0.8415
Epoch 3/7
558/558 [==============================] - 44s 79ms/step - loss: 0.5121 - accuracy: 0.8335 - val_loss: 0.4771 - val_accuracy: 0.8449
Epoch 4/7
558/558 [==============================] - 46s 82ms/step - loss: 0.4931 - accuracy: 0.8391 - val_loss: 0.4715 - val_accuracy: 0.8476
Epoch 5/7
558/558 [==============================] - 45s 81ms/step - loss: 0.4782 - accuracy: 0.8435 - val_loss: 0.4656 - val_accuracy: 0.8541
Epoch 6/7
558/558 [==============================] - 45s 81ms/step - loss: 0.4625 - accuracy: 0.8510 - val_loss: 0.4635 - val_accuracy: 0.8503
Epoch 7/7
558/558 [==============================] - 45s 81ms/step - loss: 0.4502 - accuracy: 0.8519 - val_loss: 0.4655 - val_accuracy: 0.8518

In [ ]:

model.evaluate(x_t, y_test, batch_size = 64)

39/39 [==============================] - 1s 36ms/step - loss: 0.4659 - accuracy: 0.8475


[0.46589934825897217, 0.8475191593170166]

In [ ]:
y_pred=model.predict(x_t)
y_pred=np.array([np.argmax(y) for y in y_pred])
y_pred

array([2, 1, 1, ..., 1, 1, 1])

In [ ]:
model.save('hate_speech.h5')